In [1]:
from tqdm import tqdm
from src.preprocessor import PreProcessor
from src.pipeline import IndexPipeline, QueryPipeline

with open('openai.key') as f:
    openai_key = f.read().strip()
meta_filename = 'data/StockPool.summarySource.json'

指定公司名称等信息

In [2]:
target_company_code = 'GOOG.O'
table_filenames = [
    'data/tables/APPL_IS_ADJ_Y.csv',
    'data/tables/APPL_IS_ADJ_Q.csv'
]
version = 1102

下载数据

In [3]:
data_folder = f'data/{target_company_code}_{version}'
target_db_path = f'db/{target_company_code}_{version}'
preprocessor = PreProcessor(meta_filename, target_company_code)
preprocessor.collect_data(data_folder)

Will download to: ../data/GOOG.O_1102
Collecting: news -> news -> yahoo_contetns_jp
Collecting: transcripts -> news -> transcripts
Collecting: Press Releases -> sec_gov -> 8-K_data
Collecting: news -> news -> news_contents
Collecting: news -> news -> benzinga_contents
Collecting: news -> news -> cnbc_contents
Collecting: news -> news -> news_jp_nikkei_contents
Collecting: news -> news -> wsj_contents
Collecting: research -> website -> bernste
Collecting: research -> website -> neoubs
Collecting: research -> website -> aletheia
Collecting: research -> website -> marquee
Collecting: research -> website -> jefferies
Collecting: research -> website -> jpmorgan
Collecting: research -> website -> yipit
Collecting: research -> website -> cicc
research num_docs: 91
transcripts num_docs: 5
Press Releases num_docs: 9
news num_docs: 749
tables num_docs: 0
Finish.


转换并保存数据

In [4]:
preprocessor.transform_and_save()
preprocessor.process_table(table_filenames, target_db_path, update_time="2023-11-01")

  0%|          | 0/91 [00:00<?, ?it/s]

 43%|████▎     | 39/91 [00:38<01:17,  1.48s/it]invalid pdf header: b'<div '
EOF marker not found


! Error in processing mq_fc326f94-c502-4854-8efb-5846b4e6e6df.pdf, skip it:
Stream has ended unexpectedly


 75%|███████▍  | 68/91 [01:12<00:36,  1.60s/it]invalid pdf header: b'<html'
EOF marker not found
invalid pdf header: b'<html'
EOF marker not found
invalid pdf header: b'<html'
EOF marker not found
invalid pdf header: b'<html'
EOF marker not found
invalid pdf header: b'<html'
EOF marker not found
invalid pdf header: b'<html'
EOF marker not found
invalid pdf header: b'<html'
EOF marker not found
invalid pdf header: b'<html'
EOF marker not found
invalid pdf header: b'<html'
EOF marker not found
invalid pdf header: b'<html'
EOF marker not found
invalid pdf header: b'<html'
EOF marker not found
invalid pdf header: b'<html'
EOF marker not found
invalid pdf header: b'<html'
EOF marker not found
invalid pdf header: b'<html'
EOF marker not found
invalid pdf header: b'<html'
EOF marker not found
invalid pdf header: b'<html'
EOF marker not found
invalid pdf header: b'<html'
EOF marker not found
invalid pdf header: b'<html'
EOF marker not found
 95%|█████████▍| 86/91 [01:13<00:00,  5.29it/s]invali

! Error in processing yipit_e4ec19b4-7f54-4edf-8e2b-575a7a810b78.pdf, skip it:
Stream has ended unexpectedly
! Error in processing yipit_6daeb930-e8ab-4d0f-a729-f765c99e27f2.pdf, skip it:
Stream has ended unexpectedly
! Error in processing yipit_3722cb62-0025-4349-8d82-19ea8dde6614.pdf, skip it:
Stream has ended unexpectedly
! Error in processing yipit_f3c5ba8d-6fe1-4585-86c7-7c3a70e61c73.pdf, skip it:
Stream has ended unexpectedly
! Error in processing yipit_b9e58bb1-9a12-4084-b401-b817aab75932.pdf, skip it:
Stream has ended unexpectedly
! Error in processing yipit_78388624-ea99-4ce1-b80a-4b4eb31b8f9c.pdf, skip it:
Stream has ended unexpectedly
! Error in processing yipit_60378cad-c6b9-4ad1-84e8-34c5730e3668.pdf, skip it:
Stream has ended unexpectedly
! Error in processing yipit_7c273be1-df8d-4fee-b3cb-9fc148971b63.pdf, skip it:
Stream has ended unexpectedly
! Error in processing yipit_da21cb4b-ede5-4902-9229-75c716df2227.pdf, skip it:
Stream has ended unexpectedly
! Error in processi

100%|██████████| 91/91 [01:14<00:00,  1.22it/s]

Processing tables ...
Writing to db/GOOG.O_1102\table.full_text


构建索引

In [5]:
data_filenames = preprocessor.get_data_filenames()
text_levels = preprocessor.get_text_levels()
db_paths = preprocessor.get_db_paths(target_db_path)
print(data_filenames)
print(text_levels)
print(db_paths)

['../data/GOOG.O_1102/research.json', '../data/GOOG.O_1102/news.json', '../data/GOOG.O_1102/transcripts.json', '../data/GOOG.O_1102/press.json', '../data/GOOG.O_1102/table.json']
['full_text', 'summaryText', 'summaryText', 'summaryText', 'full_text']
['db/GOOG.O_1102\\research.full_text', 'db/GOOG.O_1102\\news.summaryText', 'db/GOOG.O_1102\\transcripts.summaryText', 'db/GOOG.O_1102\\press.summaryText', 'db/GOOG.O_1102\\table.full_text']


In [6]:
for data_filename, text_level, db_path in zip(data_filenames, text_levels, db_paths):
    print("Building index from {}".format(data_filename))
    index_pipeline = IndexPipeline(data_filename, db_path, openai_key)
    index_pipeline.run(text_level, use_product_name=False)
    print('-'*40)

Building index from ../data/GOOG.O_1102/research.json
Data source:'../data/GOOG.O_1102/research.json', data name:'research', db path:'db/GOOG.O_1102\research.full_text'
File 'db/GOOG.O_1102\research.full_text/extracted_data.json' exists, skip extract data.
File 'db/GOOG.O_1102\research.full_text/embeddings.json' exists, skip create embeddings.
db path:'db/GOOG.O_1102\research.full_text'
File 'db/GOOG.O_1102\research.full_text/faiss.index' exists, skip build index.
----------------------------------------
Building index from ../data/GOOG.O_1102/news.json
Data source:'../data/GOOG.O_1102/news.json', data name:'news', db path:'db/GOOG.O_1102\news.summaryText'
File 'db/GOOG.O_1102\news.summaryText/extracted_data.json' exists, skip extract data.
File 'db/GOOG.O_1102\news.summaryText/embeddings.json' exists, skip create embeddings.
db path:'db/GOOG.O_1102\news.summaryText'
File 'db/GOOG.O_1102\news.summaryText/faiss.index' exists, skip build index.
----------------------------------------
Bu

338it [00:21, 15.71it/s]


- Done. Current Embedding cnt: 338, write to 'db/GOOG.O_1102\press.summaryText/embeddings.json'.
- Done. Final Embedding cnt: 338, write to 'db/GOOG.O_1102\press.summaryText/embeddings.json'.
db path:'db/GOOG.O_1102\press.summaryText'
Start build index ...
- Done. Embeddings cnt: 338, write to 'db/GOOG.O_1102\press.summaryText/faiss.index'.
----------------------------------------
Building index from ../data/GOOG.O_1102/table.json
Data source:'../data/GOOG.O_1102/table.json', data name:'table', db path:'db/GOOG.O_1102\table.full_text'
File 'db/GOOG.O_1102\table.full_text/extracted_data.json' exists, skip extract data.
Start create embeddings ...


21it [00:03,  6.35it/s]


- Done. Current Embedding cnt: 21, write to 'db/GOOG.O_1102\table.full_text/embeddings.json'.
- Done. Final Embedding cnt: 21, write to 'db/GOOG.O_1102\table.full_text/embeddings.json'.
db path:'db/GOOG.O_1102\table.full_text'
Start build index ...
- Done. Embeddings cnt: 21, write to 'db/GOOG.O_1102\table.full_text/faiss.index'.
----------------------------------------


测试查询

In [7]:
db_ids = [0,1,2,3]
model_ids = [1]
queries = [ 
    "What is the revenue of Google?",
]

初始化查询

In [8]:
query_pipeline = QueryPipeline(db_paths, openai_key)
query_pipeline.prepare()

db path:'db/GOOG.O_1102\research.full_text'
Prepare metadata, rawdata and index for search ...
- Done. Doc cnt: 69
db path:'db/GOOG.O_1102\news.summaryText'
Prepare metadata, rawdata and index for search ...
- Done. Doc cnt: 749
db path:'db/GOOG.O_1102\transcripts.summaryText'
Prepare metadata, rawdata and index for search ...
- Done. Doc cnt: 5
db path:'db/GOOG.O_1102\press.summaryText'
Prepare metadata, rawdata and index for search ...
- Done. Doc cnt: 9
db path:'db/GOOG.O_1102\table.full_text'
Prepare metadata, rawdata and index for search ...
- Done. Doc cnt: 2


执行查询

In [9]:
for query in queries:
    print("Query: {}".format(query))
    outputs, search_results, encode_time, search_time, respond_time = query_pipeline.query(query, 8, db_ids, model_ids)
    output = outputs[0]["summary"]
    print(output + "\n")
    print("\n【参考文档】：\n")
    for i, each in enumerate(search_results):
        title = each["title"]
        id = each["id"]
        text = each["chunk_text"].replace('\n', '')
        start_index = each["start_index"]
        print(f"【{i+1}】({title}) | 文档id：{id}，文段起始位置：{start_index}\n ... {text} ...\n")
    print(f"\n- 用户input向量化耗时：{encode_time}s\n- 向量检索耗时：{search_time}s\n- GPT总结耗时：{respond_time}s\n"+ "-"*40 + "\n")

Query: What is the revenue of Google?
谷歌（Google）的最新收入报告在文段1中，这是关于最近该季度的具体收入报告。根据文段1，谷歌服务的收入为680亿美元，其中搜索和其他广告收入为440亿美元；YouTube广告收入为80亿美元；网络广告收入为77亿美元；其他收入为83亿美元。
然而，如果你想要了解上一年度，即2022年整年的收入，我就无法提供确切的数字，因为这需要谷歌的年度报告，以上的资料中并未提供此信息。
每个季度的收入可以在文段1到文段6中找到，但请注意这并不完全等同于年度总收入，因为每个季度可能会有显著的季节性波动。（参考段落[1]）


【参考文档】：

【1】(Alphabet Inc. (GOOG) Q3 2023 Earnings Call Transcript) | 文档id：sAts_4643107，文段起始位置：21904
 ... Turning to segment results. Within Google Services, revenues were $68 billion, up 11%. Google Search and other advertising revenues of $44 billion in the quarter were up 11%, led again by growth in retail. YouTube advertising revenues of $8 billion were up 12%, driven by both brand advertising and direct response. Network advertising revenues of $7.7 billion were down 3%. Other revenues were $8.3 billion, up 21%, primarily reflecting growth in YouTube non-advertising revenues driven by subscriber growth in YouTube TV followed by YouTube Music Premium. TAC was $12.6 billion, up 7%. Google S